<a href="https://colab.research.google.com/github/JuanMaLR/bedu-equipo4-dataScience-PD/blob/main/Final_project_GA_%26_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero importamos el dataset y lo convertimos de Arff a un dataframe

In [41]:
import pandas as pd
import numpy as np
from scipy.io.arff import loadarff 
from pandas.core.strings.accessor import is_object_dtype

raw_data = loadarff('/content/Autism-Adult-Data.arff')
df = pd.DataFrame(raw_data[0])
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,b'1',b'1',b'1',b'1',b'0',b'0',b'1',b'1',b'0',b'0',...,b'f',b'White-European',b'no',b'no',b'United States',b'no',6.0,b'18 and more',b'Self',b'NO'
1,b'1',b'1',b'0',b'1',b'0',b'0',b'0',b'1',b'0',b'1',...,b'm',b'Latino',b'no',b'yes',b'Brazil',b'no',5.0,b'18 and more',b'Self',b'NO'
2,b'1',b'1',b'0',b'1',b'1',b'0',b'1',b'1',b'1',b'1',...,b'm',b'Latino',b'yes',b'yes',b'Spain',b'no',8.0,b'18 and more',b'Parent',b'YES'
3,b'1',b'1',b'0',b'1',b'0',b'0',b'1',b'1',b'0',b'1',...,b'f',b'White-European',b'no',b'yes',b'United States',b'no',6.0,b'18 and more',b'Self',b'NO'
4,b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'0',...,b'f',b'?',b'no',b'no',b'Egypt',b'no',2.0,b'18 and more',b'?',b'NO'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,b'0',b'1',b'0',b'1',b'1',b'0',b'1',b'1',b'1',b'1',...,b'f',b'White-European',b'no',b'no',b'Russia',b'no',7.0,b'18 and more',b'Self',b'YES'
700,b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',...,b'm',b'Hispanic',b'no',b'no',b'Mexico',b'no',3.0,b'18 and more',b'Parent',b'NO'
701,b'1',b'0',b'1',b'1',b'1',b'0',b'1',b'1',b'0',b'1',...,b'f',b'?',b'no',b'no',b'Russia',b'no',7.0,b'18 and more',b'?',b'YES'
702,b'1',b'0',b'0',b'1',b'1',b'0',b'1',b'0',b'1',b'1',...,b'm',b'South Asian',b'no',b'no',b'Pakistan',b'no',6.0,b'18 and more',b'Self',b'NO'


Revisamos el contenido y observamos que el dataframe tiene una b al inicio de todos los elementos de las columnas que tienen el tipo object. Esto indica que son bytes y es un problema de encoding. 

In [4]:
df.dtypes

A1_Score            object
A2_Score            object
A3_Score            object
A4_Score            object
A5_Score            object
A6_Score            object
A7_Score            object
A8_Score            object
A9_Score            object
A10_Score           object
age                float64
gender              object
ethnicity           object
jundice             object
austim              object
contry_of_res       object
used_app_before     object
result             float64
age_desc            object
relation            object
Class/ASD           object
dtype: object

Para arreglarlo creamos una función que nos permitirá remplazar de las columnas de objeto esta b con comillas. 

In [59]:
def remove_b_arff(serie):
  global df
  if is_object_dtype(serie):
    df[serie.name] = serie.str.decode('utf-8')

Lo aplicamos a todo nuestro dataframe

In [43]:
df.apply(remove_b_arff, axis = 0)
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,United States,no,6.0,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5.0,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8.0,18 and more,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,United States,no,6.0,18 and more,Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2.0,18 and more,?,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,...,f,White-European,no,no,Russia,no,7.0,18 and more,Self,YES
700,1,0,0,0,0,0,0,1,0,1,...,m,Hispanic,no,no,Mexico,no,3.0,18 and more,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,...,f,?,no,no,Russia,no,7.0,18 and more,?,YES
702,1,0,0,1,1,0,1,0,1,1,...,m,South Asian,no,no,Pakistan,no,6.0,18 and more,Self,NO


Ahora, proseguimos a revisar si existen NaNs.

In [44]:
df.isna().sum(axis=0)

A1_Score           0
A2_Score           0
A3_Score           0
A4_Score           0
A5_Score           0
A6_Score           0
A7_Score           0
A8_Score           0
A9_Score           0
A10_Score          0
age                2
gender             0
ethnicity          0
jundice            0
austim             0
contry_of_res      0
used_app_before    0
result             0
age_desc           0
relation           0
Class/ASD          0
dtype: int64

Podemos observar que existen dos NaNs en la columna age que es de tipo float. Sin embargo observando el dataset, vemos que también existen signos de interrogación en las columnas de tipo object. Por lo que habrá que remplazarlos ambos. 

En el caso de la columa age, podemos replazar los NaNs con el promedio de la columna. 

In [45]:
age_mean = df['age'].mean()
age_mean

29.698005698005698

In [46]:
df['age'] = df['age'].fillna(age_mean)
df.isna().sum(axis = 0)

A1_Score           0
A2_Score           0
A3_Score           0
A4_Score           0
A5_Score           0
A6_Score           0
A7_Score           0
A8_Score           0
A9_Score           0
A10_Score          0
age                0
gender             0
ethnicity          0
jundice            0
austim             0
contry_of_res      0
used_app_before    0
result             0
age_desc           0
relation           0
Class/ASD          0
dtype: int64

Para las columnas tipo object primero localizamos cuales son las que tienen estos signos de interrogación.

In [61]:
columns_with_question_mark = pd.Series(int)

def find_question_marks(serie):
  global columns_with_question_mark
  if is_object_dtype(serie):
    if len(list(serie.str.find('?').value_counts().index)) > 1:
      num_quesion_marks = serie.str.find('?').value_counts().values[1]
      columns_with_question_mark[serie.name] = num_quesion_marks

In [70]:
df.apply(find_question_marks, axis=0)
columns_with_question_mark
list_columns_with_question_mark = list(columns_with_question_mark.index)

In [71]:
list_columns_with_question_mark.pop(0)
list_columns_with_question_mark

['ethnicity', 'relation']

Y ahora, creamos una función para poder remplazar estos signos de interrogación con la moda de la columna. 

In [75]:
import statistics

def fill_question_mark_with_mode(serie, list_to_replace):
  global df
  if serie.name in list_to_replace:
    mode_s = statistics.mode(serie)
    df[serie.name] = serie.replace('?', mode_s)

In [78]:
df.apply(fill_question_mark_with_mode, list_to_replace=list_columns_with_question_mark, axis=0);

Volvemos a validar con nuestra función para ver si siguen existiendo columnas con signos de interrogación. Y vemos que ya no existe ninguna. 

In [79]:
columns_with_question_mark = pd.Series(int)

df.apply(find_question_marks, axis=0)
columns_with_question_mark

0    <class 'int'>
dtype: object

Si entramos al dataframe veremos que antes en la fila número 4 había un signo de interrogación en ethnicity y ahora dice White-European que es la moda. De igual modo en la columna relation de la misma fila ahora dice Self en lugar de el signo de interrogación. 

In [80]:
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,United States,no,6.0,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5.0,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8.0,18 and more,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,United States,no,6.0,18 and more,Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,White-European,no,no,Egypt,no,2.0,18 and more,Self,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,...,f,White-European,no,no,Russia,no,7.0,18 and more,Self,YES
700,1,0,0,0,0,0,0,1,0,1,...,m,Hispanic,no,no,Mexico,no,3.0,18 and more,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,...,f,White-European,no,no,Russia,no,7.0,18 and more,Self,YES
702,1,0,0,1,1,0,1,0,1,1,...,m,South Asian,no,no,Pakistan,no,6.0,18 and more,Self,NO


Observando nuevamente el dataset detectamos que la columna age_desc no agrega ningún valor a nuestro análisis ya que sólo repite la frase "18 and more" en cada fila. Por lo que la eliminaremos.

In [85]:
df['age_desc'].value_counts()

18 and more    704
Name: age_desc, dtype: int64

In [87]:
df.drop(columns='age_desc', inplace = True)

In [88]:
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,f,White-European,no,no,United States,no,6.0,Self,NO
1,1,1,0,1,0,0,0,1,0,1,24.0,m,Latino,no,yes,Brazil,no,5.0,Self,NO
2,1,1,0,1,1,0,1,1,1,1,27.0,m,Latino,yes,yes,Spain,no,8.0,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,35.0,f,White-European,no,yes,United States,no,6.0,Self,NO
4,1,0,0,0,0,0,0,1,0,0,40.0,f,White-European,no,no,Egypt,no,2.0,Self,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,f,White-European,no,no,Russia,no,7.0,Self,YES
700,1,0,0,0,0,0,0,1,0,1,34.0,m,Hispanic,no,no,Mexico,no,3.0,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,24.0,f,White-European,no,no,Russia,no,7.0,Self,YES
702,1,0,0,1,1,0,1,0,1,1,35.0,m,South Asian,no,no,Pakistan,no,6.0,Self,NO


Para poder usar nuestros datos en una red neuronal es recomendable convertirlos de string (object) a valores numéricos. Por lo que iniciaremos convirtiendo las variables dicotómicas en 1 y 0 respectivamente. 

In [99]:
#Comenzamos con gender
df['gender'].value_counts()

1    367
0    337
Name: gender, dtype: int64

In [89]:
df['gender'].replace(['f','m'],[0,1],inplace=True)
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,0,White-European,no,no,United States,no,6.0,Self,NO
1,1,1,0,1,0,0,0,1,0,1,24.0,1,Latino,no,yes,Brazil,no,5.0,Self,NO
2,1,1,0,1,1,0,1,1,1,1,27.0,1,Latino,yes,yes,Spain,no,8.0,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,35.0,0,White-European,no,yes,United States,no,6.0,Self,NO
4,1,0,0,0,0,0,0,1,0,0,40.0,0,White-European,no,no,Egypt,no,2.0,Self,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,White-European,no,no,Russia,no,7.0,Self,YES
700,1,0,0,0,0,0,0,1,0,1,34.0,1,Hispanic,no,no,Mexico,no,3.0,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,24.0,0,White-European,no,no,Russia,no,7.0,Self,YES
702,1,0,0,1,1,0,1,0,1,1,35.0,1,South Asian,no,no,Pakistan,no,6.0,Self,NO


In [100]:
#Continuamos con jundice
df['jundice'].value_counts()

0    635
1     69
Name: jundice, dtype: int64

In [90]:
df['jundice'].replace(['no','yes'],[0,1],inplace=True)
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,0,White-European,0,no,United States,no,6.0,Self,NO
1,1,1,0,1,0,0,0,1,0,1,24.0,1,Latino,0,yes,Brazil,no,5.0,Self,NO
2,1,1,0,1,1,0,1,1,1,1,27.0,1,Latino,1,yes,Spain,no,8.0,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,35.0,0,White-European,0,yes,United States,no,6.0,Self,NO
4,1,0,0,0,0,0,0,1,0,0,40.0,0,White-European,0,no,Egypt,no,2.0,Self,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,White-European,0,no,Russia,no,7.0,Self,YES
700,1,0,0,0,0,0,0,1,0,1,34.0,1,Hispanic,0,no,Mexico,no,3.0,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,24.0,0,White-European,0,no,Russia,no,7.0,Self,YES
702,1,0,0,1,1,0,1,0,1,1,35.0,1,South Asian,0,no,Pakistan,no,6.0,Self,NO


In [101]:
#Ahora con austim
df['austim'].value_counts()

0    613
1     91
Name: austim, dtype: int64

In [92]:
df['austim'].replace(['no','yes'],[0,1],inplace=True)
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,0,White-European,0,0,United States,no,6.0,Self,NO
1,1,1,0,1,0,0,0,1,0,1,24.0,1,Latino,0,1,Brazil,no,5.0,Self,NO
2,1,1,0,1,1,0,1,1,1,1,27.0,1,Latino,1,1,Spain,no,8.0,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,35.0,0,White-European,0,1,United States,no,6.0,Self,NO
4,1,0,0,0,0,0,0,1,0,0,40.0,0,White-European,0,0,Egypt,no,2.0,Self,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,White-European,0,0,Russia,no,7.0,Self,YES
700,1,0,0,0,0,0,0,1,0,1,34.0,1,Hispanic,0,0,Mexico,no,3.0,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,24.0,0,White-European,0,0,Russia,no,7.0,Self,YES
702,1,0,0,1,1,0,1,0,1,1,35.0,1,South Asian,0,0,Pakistan,no,6.0,Self,NO


In [102]:
#Ahora con used_app_before
df['used_app_before'].value_counts()

0    692
1     12
Name: used_app_before, dtype: int64

In [93]:
df['used_app_before'].replace(['no','yes'],[0,1],inplace=True)
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,0,White-European,0,0,United States,0,6.0,Self,NO
1,1,1,0,1,0,0,0,1,0,1,24.0,1,Latino,0,1,Brazil,0,5.0,Self,NO
2,1,1,0,1,1,0,1,1,1,1,27.0,1,Latino,1,1,Spain,0,8.0,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,35.0,0,White-European,0,1,United States,0,6.0,Self,NO
4,1,0,0,0,0,0,0,1,0,0,40.0,0,White-European,0,0,Egypt,0,2.0,Self,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,White-European,0,0,Russia,0,7.0,Self,YES
700,1,0,0,0,0,0,0,1,0,1,34.0,1,Hispanic,0,0,Mexico,0,3.0,Parent,NO
701,1,0,1,1,1,0,1,1,0,1,24.0,0,White-European,0,0,Russia,0,7.0,Self,YES
702,1,0,0,1,1,0,1,0,1,1,35.0,1,South Asian,0,0,Pakistan,0,6.0,Self,NO


In [103]:
#Ahora con Class/ASD
df['Class/ASD'].value_counts()

0    515
1    189
Name: Class/ASD, dtype: int64

In [94]:
#Ahora con Class/ASD
df['Class/ASD'].replace(['NO','YES'],[0,1],inplace=True)
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,0,White-European,0,0,United States,0,6.0,Self,0
1,1,1,0,1,0,0,0,1,0,1,24.0,1,Latino,0,1,Brazil,0,5.0,Self,0
2,1,1,0,1,1,0,1,1,1,1,27.0,1,Latino,1,1,Spain,0,8.0,Parent,1
3,1,1,0,1,0,0,1,1,0,1,35.0,0,White-European,0,1,United States,0,6.0,Self,0
4,1,0,0,0,0,0,0,1,0,0,40.0,0,White-European,0,0,Egypt,0,2.0,Self,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,White-European,0,0,Russia,0,7.0,Self,1
700,1,0,0,0,0,0,0,1,0,1,34.0,1,Hispanic,0,0,Mexico,0,3.0,Parent,0
701,1,0,1,1,1,0,1,1,0,1,24.0,0,White-European,0,0,Russia,0,7.0,Self,1
702,1,0,0,1,1,0,1,0,1,1,35.0,1,South Asian,0,0,Pakistan,0,6.0,Self,0


Ahora, continuaremos con las variables categóricas que tienen más de 2 categorías. En este caso son 3: ethnicity, country_of_res y relation. Como podemos ver abajo

In [95]:
df['ethnicity'].value_counts()

White-European     328
Asian              123
Middle Eastern      92
Black               43
South Asian         36
Others              30
Latino              20
Hispanic            13
Pasifika            12
Turkish              6
others               1
Name: ethnicity, dtype: int64

Pero nos damos cuenta que hay dos *others*, sólo que uno con mayúscula la letra O y otra con minúscula. Cambiaremos la que tiene letra minúscula por mayúscula para agruparlos a ambos. 

In [114]:
df['ethnicity'].replace('others', 'Others', inplace=True)

In [115]:
df['ethnicity'].value_counts()

White-European     328
Asian              123
Middle Eastern      92
Black               43
South Asian         36
Others              31
Latino              20
Hispanic            13
Pasifika            12
Turkish              6
Name: ethnicity, dtype: int64

In [116]:
#Mapeamos ethnicity
eth_list = list(df['ethnicity'].value_counts().index)

In [122]:
eth_rep_list = list(range(0, len(eth_list)))
eth_rep_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [123]:
df['ethnicity'].replace(eth_list,eth_rep_list,inplace=True)

In [124]:
df['ethnicity'].value_counts()

0    328
1    123
2     92
3     43
4     36
5     31
6     20
7     13
8     12
9      6
Name: ethnicity, dtype: int64

In [97]:
df['contry_of_res'].value_counts()

United States           113
United Arab Emirates     82
New Zealand              81
India                    81
United Kingdom           77
                       ... 
China                     1
Chile                     1
Lebanon                   1
Burundi                   1
Cyprus                    1
Name: contry_of_res, Length: 67, dtype: int64

In [125]:
#Mapeamos contry_of_res
country_list = list(df['contry_of_res'].value_counts().index)
country_list

['United States',
 'United Arab Emirates',
 'New Zealand',
 'India',
 'United Kingdom',
 'Jordan',
 'Australia',
 'Canada',
 'Sri Lanka',
 'Afghanistan',
 'France',
 'Netherlands',
 'Brazil',
 'Mexico',
 'Russia',
 'Iran',
 'Ireland',
 'Malaysia',
 'Viet Nam',
 'Italy',
 'Germany',
 'Austria',
 'Philippines',
 'Saudi Arabia',
 'Kazakhstan',
 'Belgium',
 'Romania',
 'Spain',
 'Bangladesh',
 'Pakistan',
 'Egypt',
 'Sweden',
 'AmericanSamoa',
 'Ethiopia',
 'South Africa',
 'Ukraine',
 'Argentina',
 'Bahamas',
 'Armenia',
 'Iceland',
 'Aruba',
 'Niger',
 'Bolivia',
 'Czech Republic',
 'Finland',
 'Indonesia',
 'Turkey',
 'Iraq',
 'Azerbaijan',
 'Angola',
 'Nepal',
 'Ecuador',
 'Sierra Leone',
 'Portugal',
 'Serbia',
 'Uruguay',
 'Japan',
 'Hong Kong',
 'Nicaragua',
 'Costa Rica',
 'Oman',
 'Tonga',
 'China',
 'Chile',
 'Lebanon',
 'Burundi',
 'Cyprus']

In [126]:
country_rep_list = list(range(0, len(country_list)))
country_rep_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66]

In [127]:
df['contry_of_res'].replace(country_list,country_rep_list,inplace=True)

In [128]:
df['contry_of_res'].value_counts()

0     113
1      82
2      81
3      81
4      77
     ... 
62      1
63      1
64      1
65      1
66      1
Name: contry_of_res, Length: 67, dtype: int64

In [98]:
df['relation'].value_counts()

Self                        617
Parent                       50
Relative                     28
Others                        5
Health care professional      4
Name: relation, dtype: int64

In [129]:
#Mapeamos relation
relation_list = list(df['relation'].value_counts().index)
relation_list

['Self', 'Parent', 'Relative', 'Others', 'Health care professional']

In [130]:
relation_rep_list = list(range(0, len(relation_list)))
relation_rep_list

[0, 1, 2, 3, 4]

In [131]:
df['relation'].replace(relation_list,relation_rep_list,inplace=True)

In [132]:
df['relation'].value_counts()

0    617
1     50
2     28
3      5
4      4
Name: relation, dtype: int64

Por lo que al final tenemos el siguiente mapeo de variables: 
- gender
  - m = 1
  - f = 0
- jundice, autism, used_app_before, Class/ASD
  - no = 0
  - yes = 1
- ethnicity
  - White-European = 0
  - Asian = 1
  - Middle Eastern = 2
  - Black = 3
  - South Asian = 4
  - Others = 5
  - Latino = 6
  - Hispanic = 7
  - Pasifika = 8
  - Turkish = 9
- contry_of_res
  - United States = 0
  - ...
  - Cyprus = 66
- relation
  - Self = 0
  - Parent = 1
  - Relative = 2
  - Others = 3
  - Health care professional = 4

Por último, revisamos que ahora todas nuestras columnas sean de valores numéricos

In [134]:
df.dtypes

A1_Score            object
A2_Score            object
A3_Score            object
A4_Score            object
A5_Score            object
A6_Score            object
A7_Score            object
A8_Score            object
A9_Score            object
A10_Score           object
age                float64
gender               int64
ethnicity            int64
jundice              int64
austim               int64
contry_of_res        int64
used_app_before      int64
result             float64
relation             int64
Class/ASD            int64
dtype: object

Nos damos cuenta que hay algunos datos flotantes pero la realidad es que no aporta nada el tenerlos con coma flotante. Podríamos cambiarlos a enteros para así ahorrar espacio en memoria. Y los datos tipo object también pueden convertirse a int ya que son binarios. 

In [135]:
df = df.astype(int)

In [136]:
df.dtypes

A1_Score           int64
A2_Score           int64
A3_Score           int64
A4_Score           int64
A5_Score           int64
A6_Score           int64
A7_Score           int64
A8_Score           int64
A9_Score           int64
A10_Score          int64
age                int64
gender             int64
ethnicity          int64
jundice            int64
austim             int64
contry_of_res      int64
used_app_before    int64
result             int64
relation           int64
Class/ASD          int64
dtype: object

De igual modo es importante renombrar las columnas para poder entender mejor a qué se refiere cada una y para poder aplicar nuestro algoritmo genético correctamente. Además que los nombres originales continen errores de ortografía.

In [133]:
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,26.0,0,0,0,0,0,0,6.0,0,0
1,1,1,0,1,0,0,0,1,0,1,24.0,1,6,0,1,12,0,5.0,0,0
2,1,1,0,1,1,0,1,1,1,1,27.0,1,6,1,1,27,0,8.0,1,1
3,1,1,0,1,0,0,1,1,0,1,35.0,0,0,0,1,0,0,6.0,0,0
4,1,0,0,0,0,0,0,1,0,0,40.0,0,0,0,0,30,0,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25.0,0,0,0,0,14,0,7.0,0,1
700,1,0,0,0,0,0,0,1,0,1,34.0,1,7,0,0,13,0,3.0,1,0
701,1,0,1,1,1,0,1,1,0,1,24.0,0,0,0,0,14,0,7.0,0,1
702,1,0,0,1,1,0,1,0,1,1,35.0,1,4,0,0,29,0,6.0,0,0


In [140]:
df = df.rename({"A1_Score": "s1",
            "A2_Score": "s2",
            "A3_Score": "s3",
            "A4_Score": "s4",
            "A5_Score": "s5",
            "A6_Score": "s6",
            "A7_Score": "s7",
            "A8_Score": "s8",
            "A9_Score": "s9",
            "A10_Score": "s10",
            "age": "age",
            "gender": "gender",
            "ethnicity": "ethnicity",
            "jundice": "jaundice",
            "austim": "autism",
            "contry_of_res": "country_of_residence",
            "used_app_before":"used_app_before",
            "result":"score",
            "relation":"test_answered_by",
            "Class/ASD":"family_member_with_PDD"},
            axis = 1)

Por último, vemos que hay ciertas columnas que no nos van a servir para poder predecir si una persona debe ir a diagnóstico porque podría tener autismo. Como por ejemplo, si ha usado la app en el pasado o quién contestó el test. Entonces estas columnas deben ser eliminadas.

In [141]:
df

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,age,gender,ethnicity,jaundice,autism,country_of_residence,used_app_before,score,test_answered_by,family_member_with_PDD
0,1,1,1,1,0,0,1,1,0,0,26,0,0,0,0,0,0,6,0,0
1,1,1,0,1,0,0,0,1,0,1,24,1,6,0,1,12,0,5,0,0
2,1,1,0,1,1,0,1,1,1,1,27,1,6,1,1,27,0,8,1,1
3,1,1,0,1,0,0,1,1,0,1,35,0,0,0,1,0,0,6,0,0
4,1,0,0,0,0,0,0,1,0,0,40,0,0,0,0,30,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0,1,0,1,1,0,1,1,1,1,25,0,0,0,0,14,0,7,0,1
700,1,0,0,0,0,0,0,1,0,1,34,1,7,0,0,13,0,3,1,0
701,1,0,1,1,1,0,1,1,0,1,24,0,0,0,0,14,0,7,0,1
702,1,0,0,1,1,0,1,0,1,1,35,1,4,0,0,29,0,6,0,0
